In [69]:
import os
import time
import json
import requests
import tempfile
import numpy as np
import pandas as pd

import boto3
import sagemaker
from sagemaker.inputs import TrainingInput, CreateModelInput
from sagemaker.workflow.steps import TrainingStep, CreateModelStep, ProcessingStep, TransformStep

from sagemaker.estimator import Estimator
from sagemaker import get_execution_role

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import ParameterString
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingOutput, ProcessingInput

from time import gmtime, strftime

In [70]:
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)
sagemaker_boto_client = boto_session.client("sagemaker")

# sagemaker_session = sagemaker.session.Session()
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_boto_client
)

account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]
# region = sagemaker_session.boto_session.region_name
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)

sm_role = get_execution_role()

In [71]:
# s3_client.list_buckets()
# sm_client.list_model_package_groups()

In [72]:
# S3 prefix
bucket = "artwork-content-trial-bucket"
prefix="mch-artwork-content"
train_data_dir_prefix="data"
pipeline_dir_prefix="pipeline-data"
training_input_prefix = "content_ovr_anonymised.csv"

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="Approved"
)


In [73]:
training_input = f"s3://{bucket}/{train_data_dir_prefix}/{training_input_prefix}"


# input_train = ParameterString(
#     name="TrainData",
#     default_value=training_input,
# )

# model_output = ParameterString(name="ModelOutput", default_value=f"s3://{bucket}/model")

In [74]:
train_image_uri = '791574662255.dkr.ecr.us-east-1.amazonaws.com/artwork-content-repo:latest'
estimator_output_path = f"s3://{bucket}/{pipeline_dir_prefix}/"

In [75]:
source_dir = os.path.realpath('scoring/')

content_estimator = Estimator(
    image_uri=train_image_uri,
    instance_type="ml.m5.xlarge",
    instance_count=1,
#     entry_point='inference.py',
#     source_dir=source_dir,
    output_path=estimator_output_path,
    role=sm_role,
)

In [76]:
step_train = TrainingStep(
    name="TrainingStep",
    estimator=content_estimator,
    inputs=training_input,
)

In [77]:
mdl_name="artwork-content-model"
# source_dir = os.path.realpath('scoring/')
# code_location = f"s3://{bucket}/code"

model = Model(
    name=mdl_name,
    image_uri=content_estimator.training_image_uri(),
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
#     entry_point='inference.py',
#     source_dir=source_dir,
    sagemaker_session=sagemaker_session,
    role=sm_role,
)

inputs = CreateModelInput(instance_type="ml.m5.xlarge")

step_model_create = CreateModelStep(name="CreateModelStep", model=model, inputs=inputs)

In [78]:
# timestamp = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
# mpg_name = f"artwork-content-{timestamp}"
mpg_name = "MCH-Content-Models"

step_model_registration = RegisterModel(
    name="RegisterModelStep",
    estimator=content_estimator,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=mpg_name,
    approval_status=model_approval_status,
)

In [79]:
# s3_client.upload_file(Filename="deploy_3.py", Bucket=bucket, Key="code/deploy_3.py")
s3_client.upload_file(Filename="scoring/inference.py", Bucket=bucket, Key="code/scoring/inference.py")

# deploy_script_uri = f"s3://{bucket}/code/deploy_3.py"

endpoint_name = "mch-artwork-content-ep-1"

# source_dir = os.path.realpath('scoring/')
# model_version_arn = sm_client.list_model_packages(ModelPackageGroupName=mpg_name)['ModelPackageSummaryList'][0]['ModelPackageArn']

inference_image_uri = '791574662255.dkr.ecr.us-east-1.amazonaws.com/artwork-content-inference:latest'

# endpoint_name = 'DEMO-modelregistry-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

deployment_processor = SKLearnProcessor(
    framework_version="1.0-1",
    role=sm_role,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name=f"{prefix}-deploy",
    sagemaker_session=sagemaker_session,
)

step_deploy_model = ProcessingStep(
    name="DeployContentModel",
    processor=deployment_processor,
    inputs=[
        ProcessingInput(
            source=f's3://{bucket}/code/scoring/inference.py',
            destination='/opt/ml/processing/input',
            input_name='scoring_input'
        )
    ],
    job_arguments=[
        "--train_image_uri",
        train_image_uri,
        "--model_data",
        step_train.properties.ModelArtifacts.S3ModelArtifacts,
        "--model_name",
        step_model_create.properties.ModelName,
        "--sm_role",
        sm_role,
        "--region",
        region,
        "--endpoint-name",
        endpoint_name,
#         "--model_version_arn",
#         model_version_arn,
#         "--inference_image_uri",
#         inference_image_uri
    ],
    code='deploy_3.py',
    outputs=[
        ProcessingOutput(output_name="endpoint_arn", source="/opt/ml/processing/endpoint_arn")
    ],
)

In [80]:
pipeline_name = f"artwork-content-pipeline-demo"

pipeline_steps = [step_train, step_model_create, step_model_registration, step_deploy_model]

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[model_approval_status],
    steps=pipeline_steps,
    sagemaker_session=sagemaker_session
)

In [81]:
pipeline.upsert(role_arn=sm_role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:791574662255:pipeline/artwork-content-pipeline-demo',
 'ResponseMetadata': {'RequestId': 'ed6f6121-5e02-4941-8055-b35b37b6eaa8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ed6f6121-5e02-4941-8055-b35b37b6eaa8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Mon, 16 Jan 2023 14:04:39 GMT'},
  'RetryAttempts': 0}}

In [82]:
execution = pipeline.start()
execution.wait()
execution.describe()

WaiterError: Waiter PipelineExecutionComplete failed: Waiter encountered a terminal failure state: For expression "PipelineExecutionStatus" we matched expected path: "Failed"

In [ ]:
# pipeline.delete()

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker_session)
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)